In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import joblib

In [ ]:
ride_share_df = pd.read_csv("/content/drive/MyDrive/ML Project/rideshare_kaggle.csv")

In [ ]:
ride_share_df.columns[ride_share_df.isnull().any()]

Index(['price'], dtype='object')

In [ ]:
ride_share_df['price'] = ride_share_df['price'].fillna(ride_share_df['price'].mean())

In [ ]:
ride_share_df.columns[ride_share_df.isnull().any()]

Index([], dtype='object')

In [ ]:
loc_coord = {'Haymarket Square' : [42.3628, -71.0583],
             'North Station' : [42.3664, -71.0620],
             'Northeastern University' : [42.3398, -71.0892],
             'West End' : [42.4556, -75.0966],
             'South Station' : [42.3519, -71.0552],
             'Fenway' : [42.3519, -71.0552],
             'Theatre District' : [40.7590, -73.9845],
             'Beacon Hill' : [42.3588, -71.0707],
             'Back Bay' : [42.3503, -71.0810],
             'North End' : [42.3647, -71.0542],
             'Financial District' : [40.7077, -74.0083],
             'Boston University' : [42.3505, -71.1054]
}

In [ ]:
src_latitude = []
src_longitude = []
dest_latitude = []
dest_longitude = []

for i in range(len(ride_share_df)):
    src_latitude.append(loc_coord[ride_share_df['source'][i]][0])
    src_longitude.append(loc_coord[ride_share_df['source'][i]][1])
    dest_latitude.append(loc_coord[ride_share_df['destination'][i]][0])
    dest_longitude.append(loc_coord[ride_share_df['destination'][i]][1])

ride_share_df['pickup_latitude'] = src_latitude
ride_share_df['pickup_longitude'] = src_longitude
ride_share_df['dropoff_latitude'] = dest_latitude
ride_share_df['dropoff_longitude'] = dest_longitude

In [ ]:
rel_cols = ['datetime', 'hour', 'day', 'month', 'source', 'destination', 'distance', 'price', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']

In [ ]:
new_ride_share = ride_share_df[rel_cols]
new_ride_share['datetime'] = new_ride_share['datetime'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
new_ride_share

,datetime,hour,day,month,source,destination,distance,price,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,2018-12-16 09:30:07,9,16,12,Haymarket Square,North Station,0.44,5.000000,42.3628,-71.0583,42.3664,-71.0620
1,2018-11-27 02:00:23,2,27,11,Haymarket Square,North Station,0.44,11.000000,42.3628,-71.0583,42.3664,-71.0620
2,2018-11-28 01:00:22,1,28,11,Haymarket Square,North Station,0.44,7.000000,42.3628,-71.0583,42.3664,-71.0620
3,2018-11-30 04:53:02,4,30,11,Haymarket Square,North Station,0.44,26.000000,42.3628,-71.0583,42.3664,-71.0620
4,2018-11-29 03:49:20,3,29,11,Haymarket Square,North Station,0.44,9.000000,42.3628,-71.0583,42.3664,-71.0620
...,...,...,...,...,...,...,...,...,...,...,...,...
693066,2018-12-01 23:53:05,23,1,12,West End,North End,1.00,13.000000,42.4556,-75.0966,42.3647,-71.0542
693067,2018-12-01 23:53:05,23,1,12,West End,North End,1.00,9.500000,42.4556,-75.0966,42.3647,-71.0542
693068,2018-12-01 23:53:05,23,1,12,West End,North End,1.00,16.545125,42.4556,-75.0966,42.3647,-71.0542
693069,2018-12-01 23:53:05,23,1,12,West End,North End,1.00,27.000000,42.4556,-75.0966,42.3647,-71.0542


In [ ]:
#separate the other attributes from the predicting attribute
x = new_ride_share.drop("price",axis=1)
#separte the predicting attribute into Y for model training 
y = new_ride_share["price"]

In [ ]:
from sklearn import  preprocessing
label_encoder = preprocessing.LabelEncoder()
for i in x.columns:
  x[i] = label_encoder.fit_transform(x[i])

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
x = pd.DataFrame(StandardScaler().fit_transform(x),columns = x.columns)

In [ ]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
x_train

,datetime,hour,day,month,source,destination,distance,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
295098,0.382709,1.637980,-1.026695,0.839341,1.015350,1.305202,-0.813027,0.004621,1.138871,-1.315293,-1.030831
75594,0.122632,-0.952653,-1.422115,0.839341,-0.433915,-0.144089,-0.975350,-1.645133,-1.340772,0.664420,0.828885
41271,0.265586,1.062284,-1.422115,0.839341,1.015350,1.305202,-0.903207,0.004621,1.138871,-1.315293,-1.030831
304967,-1.100987,-0.376957,0.950409,-1.191411,-0.433915,-0.144089,-0.840081,-1.645133,-1.340772,0.664420,0.828885
117673,1.792852,-0.089109,0.357278,0.839341,-1.593327,1.015344,-0.758919,-0.655280,-0.100951,0.004516,1.138838
...,...,...,...,...,...,...,...,...,...,...,...
259178,0.783147,0.486587,-0.433564,0.839341,-0.723768,-0.433947,2.000575,0.004621,1.138871,-1.645245,-1.340783
365838,-1.374965,-1.096577,0.554988,-1.191411,-0.144062,-0.433947,-1.146691,0.664523,0.828915,-1.645245,-1.340783
131932,-0.457900,1.494056,1.345830,-1.191411,1.015350,-0.433947,-1.381158,0.004621,1.138871,-1.645245,-1.340783
671155,1.260249,-0.233033,-0.038143,0.839341,1.595056,0.725486,0.810205,1.654375,-1.650727,-0.985340,-0.410926


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
def model_score(x_train,y_train,x_test,y_test):
    neigh = KNeighborsRegressor(n_neighbors=39)
    neigh.fit(x_train, y_train)
    y_pred_sk_train = neigh.predict(x_test)
    return np.sqrt(mean_squared_error(y_test,y_pred_sk_train))

In [ ]:
score = model_score(x_train,y_train,x_test,y_test)

In [ ]:
x_train_arr = np.asarray(x_train)
x_test_arr = np.asarray(x_test)

In [ ]:
# x_test

In [ ]:
x_train_arr.shape

(554456, 11)

In [ ]:
actual_score = model_score(x_train_arr, y_train, x_test_arr, y_test)

In [ ]:
print('New Testing Accuracy before ablation is - '+str(actual_score))

New Testing Accuracy before ablation is - 8.516567138235667


In [ ]:
scores = []
for i in range(x_train_arr.shape[1]):
    # use_column = [ndx != i for ndx in range(x_train.shape[1])]
    # for j in range(x_train.shape[1]):
    #   print(use_column[j])
    #   if(use_column[j]=='False'):
    # use_column = [ndx != i for ndx in range(x_train_arr.shape[1])]
    # print(use_column[1])
    # if(use_column[j] == 'False'):
    scores.append(model_score(x_train_arr[:, i:i+1],np.asarray(y_train),x_test_arr[:, i:i+1],np.asarray(y_test)))

In [ ]:
scores

[9.044978874053708,
 9.010911865730916,
 9.035990061034907,
 8.99514458216295,
 9.109446721361987,
 8.917660729267974,
 8.511391334726163,
 9.029166358427448,
 9.117931567032969,
 8.921407245892823,
 8.93740037693995]

In [ ]:
sorted(enumerate([actual_score - s for s in scores]), key=lambda ndx_score: ndx_score[1], reverse=True)[:5]

[(6, 0.005175803509503751),
 (5, -0.401093591032307),
 (9, -0.40484010765715617),
 (10, -0.420833238704283),
 (3, -0.47857744392728385)]

In [ ]:
# dic[0][0]

In [ ]:
# new_X_train = np.array([])
# new_X_test = np.array([])
# for i in range(len(dic)):
#   j = dic[i][0]
#   print(j)
#   new_X_train = np.hstack((new_X_train,x_train_arr[:,j:j+1]))
#   new_X_test = np.hstack((new_X_test,x_train_arr[:,j:j+1]))

In [ ]:
new_x_train = np.hstack((x_train_arr[:,6:7],x_train_arr[:,5:6],x_train_arr[:,9:10],x_train_arr[:,10:11],x_train_arr[:,3:4]))
new_x_test = np.hstack((x_test_arr[:,6:7],x_test_arr[:,5:6],x_test_arr[:,9:10],x_test_arr[:,10:11],x_test_arr[:,3:4]))

In [ ]:
# new_X_train

In [ ]:
actual_score_new_train = model_score(new_x_train, y_train, new_x_train, y_train)
actual_score_new_train

8.424698837963945

In [ ]:
print('New Training Accuracy after ablation is - '+str(actual_score_new_train))

New Training Accuracy after ablation is - 8.424698837963945


In [ ]:
actual_score_new_test = model_score(new_x_train, y_train, new_x_test, y_test)

In [ ]:
print('New Testing Accuracy after ablation is - '+str(actual_score_new_test))

New Testing Accuracy after ablation is - 8.470993490676369
